# Settings

In [1]:
# Desired percentage of NIL mentions
perc_nil = 0.7

# Number of batch to create
n_batch = 10

# Desired NIL mentions per batch (in dev and test)
desired_nil = 20

# Desired size of a single dev and test batch
dev_test_desired_batch_size = 100

# Random state
random_state = 1234

# Output dir
outdir = 'incremental_dataset'

# Imports

In [2]:
import pandas as pd
from tqdm import tqdm,trange
from glob import glob
import numpy as np
from sklearn.model_selection import StratifiedKFold
import os
import json

# Download

Download the original dataset from https://github.com/yasumasaonoe/ET4EL

# Extract it

```
tar -xzf unseen_mentions.tar.gz
```

You should be able to see a folder named `unseen_mentions` with the following content:
```
unseen_mentions/
unseen_mentions/dev.json
unseen_mentions/test.json
unseen_mentions/train
unseen_mentions/train/train_2.json
unseen_mentions/train/train_1.json
unseen_mentions/train/train_4.json
unseen_mentions/train/train_3.json
unseen_mentions/train/train_5.json
unseen_mentions/train/train_0.json
```

# Load Original Dataset

In [3]:
df = pd.read_json("./msnbc-test-kilt.jsonl", lines=True)
df["answer"] = df["output"].apply(lambda x: x[0]["answer"])

# Proceed

### Calculate the absolute frequency with which each entity is mentioned

In [4]:
mention_frequency = df.groupby("answer").size()

In [5]:
mention_frequency

answer
'N Sync                                    2
Al Arabiya                                 1
Al Jazeera                                 2
Alabama                                    6
Alabama Crimson Tide                       4
                                          ..
Women's National Basketball Association    2
World Trade Center                         1
Wyoming                                    1
Xinhua News Agency                         1
Yamuna                                     1
Length: 288, dtype: int64

In [6]:
freq_df = pd.DataFrame(mention_frequency)
freq_df.columns = ['freq']
freq_df.shape

(288, 1)

In [7]:
freq_df.head()

,freq
answer,
'N Sync,2
Al Arabiya,1
Al Jazeera,2
Alabama,6
Alabama Crimson Tide,4


In [8]:
# calculate the median frequency with which all the entities are mentioned
med_freq = np.median(freq_df['freq'])
med_freq

1.0

Calculate the probability that each entity is NIL

In [9]:
np.random.seed(random_state)

freq_df['p_formula'] = perc_nil ** (freq_df['freq'] / med_freq)
s = np.random.uniform(0, 1, freq_df.shape[0])
freq_df['p_uniform'] = s
freq_df.head()

,freq,p_formula,p_uniform
answer,,,
'N Sync,2,0.490000,0.191519
Al Arabiya,1,0.700000,0.622109
Al Jazeera,2,0.490000,0.437728
Alabama,6,0.117649,0.785359
Alabama Crimson Tide,4,0.240100,0.779976


In [10]:
freq_df['NIL'] = freq_df['p_uniform'] < freq_df['p_formula']
freq_df.head()

,freq,p_formula,p_uniform,NIL
answer,,,,
'N Sync,2,0.490000,0.191519,True
Al Arabiya,1,0.700000,0.622109,True
Al Jazeera,2,0.490000,0.437728,True
Alabama,6,0.117649,0.785359,False
Alabama Crimson Tide,4,0.240100,0.779976,False


In [11]:
print('N# NIL:', freq_df.eval('NIL').sum(), 'percentage:', freq_df.eval('NIL').sum() / freq_df.shape[0] * 100, '%')

N# NIL: 164 percentage: 56.94444444444444 %


In [12]:
# Setting NIL freq to 0 so that when we split in batches each batch has the same number of NILs
freq_df.loc[freq_df['NIL'] == True, 'freq'] = 0

In [14]:
train_df_merged = df.join(freq_df, how='left', on='answer')

In [17]:
train_df_merged.head(20)

,id,input,output,meta,candidates,answer,freq,p_formula,p_uniform,NIL
0,0,Stocks end back and forth day slightly higher ...,"[{'answer': 'New York City', 'provenance': [{'...",{'left_context': 'Stocks end back and forth da...,"[New York, New York City, New York (magazine),...",New York City,6,0.117649,0.734945,False
1,1,Stocks end back and forth day slightly higher ...,"[{'answer': 'New York Stock Exchange', 'proven...",{'left_context': 'Stocks end back and forth da...,"[New York Stock Exchange, Wall Street bombing]",New York Stock Exchange,3,0.343000,0.801603,False
2,2,Stocks end back and forth day slightly higher ...,"[{'answer': 'NASDAQ', 'provenance': [{'title':...",{'left_context': 'Stocks end back and forth da...,[NASDAQ],NASDAQ,2,0.490000,0.890557,False
3,3,day slightly higher NEW YORK Stocks finished a...,"[{'answer': 'United States', 'provenance': [{'...",{'left_context': 'day slightly higher NEW YORK...,"[United States, United States Reports, Billboa...",United States,14,0.006782,0.658873,False
4,4,dollar after three down days kept investors ap...,"[{'answer': 'The Home Depot', 'provenance': [{...",{'left_context': 'dollar after three down days...,[The Home Depot],The Home Depot,20,0.000798,0.598237,False
5,5,down days kept investors appetite for stocks i...,"[{'answer': 'Saks, Inc.', 'provenance': [{'tit...",{'left_context': 'down days kept investors app...,"[Saks, Inc.]","Saks, Inc.",2,0.490000,0.630314,False
6,6,investors appetite for stocks in check An eigh...,"[{'answer': 'Target Corporation', 'provenance'...",{'left_context': 'investors appetite for stock...,[Target Corporation],Target Corporation,2,0.490000,0.901140,False
7,7,returns in stocks and commodities lifting shar...,"[{'answer': 'ING Group', 'provenance': [{'titl...",{'left_context': 'returns in stocks and commod...,[ING Group],ING Group,1,0.700000,0.767117,False
8,8,commodities lifting share prices Trading volum...,"[{'answer': 'New York City', 'provenance': [{'...",{'left_context': 'commodities lifting share pr...,"[New York, New York City, New York (magazine),...",New York City,6,0.117649,0.734945,False
9,9,of the holiday shopping season Despite the dra...,"[{'answer': 'Federal Reserve System', 'provena...",{'left_context': 'of the holiday shopping seas...,"[Federal Reserve System, The Fed (Columbia new...",Federal Reserve System,0,0.700000,0.123943,True


In [18]:
print('Percentage of NIL mentions in train:', train_df_merged.eval('NIL').sum() / train_df_merged.shape[0] * 100)

Percentage of NIL mentions in train: 37.19512195121951


In [34]:
final_df = pd.DataFrame()
results = []
for index, row in train_df_merged.iterrows():
    if row["NIL"] == True:
        candidates = row["candidates"]
        filtered_cands = list(
            filter(
                lambda x: row["answer"] not in x,
                candidates,
            )
        )
        row["candidates"] = filtered_cands
        row["output"][0]["answer"] = "Not In Candidates"
        results.append(row.to_dict())
    else:
        results.append(row.to_dict())

In [ ]:
#write to jsonl file
with open(outdir + '/msnbc-test-kilt-nil.jsonl', 'w') as outfile:
    for entry in results:
        json.dump(entry, outfile)
        outfile.write('\n')

In [80]:
train_df_merged.to_csv(os.path.join(outdir, 'msnbc_test_nilled_good.csv'), index=False)

# Propagate NIL mentions in test and dev

In [ ]:
dev_df_merged = dev_df.join(freq_df, how='left', on='wikiId')

In [ ]:
dev_df_merged['p_formula'] = dev_df_merged['p_formula'].fillna(-1)
dev_df_merged['p_uniform'] = dev_df_merged['p_uniform'].fillna(-1)
dev_df_merged['NIL'] = dev_df_merged['NIL'].fillna(False)

Compute entity frequencies in dev for a correct sampling

In [ ]:
mention_frequency_dev = dev_df_merged.query('~NIL').groupby('wikiId').size()

In [ ]:
mention_frequency_dev

In [ ]:
freq_df_dev = pd.DataFrame(mention_frequency_dev)
freq_df_dev.columns = ['freq']
freq_df_dev.shape

In [ ]:
freq_df_dev.head()

In [ ]:
dev_df_merged = dev_df_merged.drop(columns=['freq']).join(freq_df_dev, how='left', on='wikiId')

In [ ]:
# set NIL entities freq to 0 for stratifying the NIL class
dev_df_merged.loc[dev_df_merged['NIL'], 'freq'] = 0

In [ ]:
dev_df_merged['freq'] = dev_df_merged['freq'].astype(int)

In [ ]:
assert not dev_df_merged['NIL'].isna().any()

In [ ]:
print('dev NIL N#:', dev_df_merged.eval('NIL').sum(), 'Percentage:', dev_df_merged.eval('NIL').sum() / dev_df_merged.shape[0] * 100)

In [ ]:
test_df_merged = test_df.join(freq_df, how='left', on='wikiId')

In [ ]:
test_df_merged['p_formula'] = test_df_merged['p_formula'].fillna(-1)
test_df_merged['p_uniform'] = test_df_merged['p_uniform'].fillna(-1)
test_df_merged['NIL'] = test_df_merged['NIL'].fillna(False)

Compute entity frequencies in test for a correct sampling

In [ ]:
mention_frequency_test = test_df_merged.query('~NIL').groupby('wikiId').size()

In [ ]:
mention_frequency_test

In [ ]:
freq_df_test = pd.DataFrame(mention_frequency_test)
freq_df_test.columns = ['freq']
freq_df_test.shape

In [ ]:
freq_df_test.head()

In [ ]:
test_df_merged = test_df_merged.drop(columns=['freq']).join(freq_df_test, how='left', on='wikiId')

In [ ]:
# set NIL entities freq to 0 for stratifying the NIL class
test_df_merged.loc[test_df_merged['NIL'], 'freq'] = 0

In [ ]:
test_df_merged['freq'] = test_df_merged['freq'].astype(int)

In [ ]:
assert not test_df_merged['NIL'].isna().any()

In [ ]:
print('test NIL N#:', test_df_merged.eval('NIL').sum(), 'Percentage:', test_df_merged.eval('NIL').sum() / test_df_merged.shape[0] * 100)

# Transplant

In [ ]:
# I want 500 NIL in dev and test
# I want 100k total in both dev and test

In [ ]:
train_unique = train_df[['word', 'wikiId']].drop_duplicates(keep='first')

In [ ]:
dev_unique = dev_df[['word', 'wikiId']].drop_duplicates(keep='first')

In [ ]:
test_unique = test_df[['word', 'wikiId']].drop_duplicates(keep='first')

In [ ]:
# ensure no duplicates
assert not train_unique.duplicated().any()
assert not dev_unique.duplicated().any()
assert not test_unique.duplicated().any()

In [ ]:
# ensure unseen mentions holds
assert not pd.concat([train_unique, dev_unique]).duplicated().any()
assert not pd.concat([train_unique, test_unique]).duplicated().any()
assert not pd.concat([dev_unique, test_unique]).duplicated().any()

## NIL

In [ ]:
# prendo n menzioni NIL dal train che non siano nel dev/test e ce le metto

In [ ]:
train_df_nil = train_df_merged.query('NIL')

In [ ]:
desired_nil_total = desired_nil * n_batch
nil_to_add_dev = desired_nil_total - dev_df_merged.eval('NIL').sum()
nil_to_add_dev

In [ ]:
nil_to_add_test = desired_nil_total - test_df_merged.eval('NIL').sum()
nil_to_add_test

In [ ]:
def get_subset_sum_ge(df, sum_, random_state):
    """
    Gets a subset whose sum is greater than sum_
    """
    
    #print(sum_)
    
    if df.shape[0] == 0:
        return None
    
    # get a sample
    item = df.sample(n=1, random_state=random_state)
    random_state += 1
    df = df.drop(item.index) # remove just extracted item
    
    subset = item.index

    current_sum = item['count'].values[0]
    while current_sum < sum_:
        print(f'\r{current_sum}/{sum_}',end='')
        # get a sample
        item = df.sample(n=1, random_state=random_state)
        random_state += 1
        df = df.drop(item.index) # remove just extracted item

        current_sum += item['count'].values[0]
        
        subset = subset.union(item.index)
    print()
    return subset

In [ ]:
train_df_nil_unique_count = pd.DataFrame(
    train_df_nil[['wikiId', 'word']].value_counts(), columns=['count']).reset_index()
train_df_nil_unique_count.head()

In [ ]:
sample_transplant_to_dev = get_subset_sum_ge(train_df_nil_unique_count, nil_to_add_dev, random_state)
print(train_df_nil_unique_count.loc[sample_transplant_to_dev]['count'].sum(), '>=', nil_to_add_dev)
assert train_df_nil_unique_count.loc[sample_transplant_to_dev]['count'].sum() >= nil_to_add_dev

In [ ]:
# remove sample_transplant to dev before extracting a sample for test
train_df_nil_unique_count_no_dev = train_df_nil_unique_count.drop(sample_transplant_to_dev)
sample_transplant_to_test = get_subset_sum_ge(train_df_nil_unique_count_no_dev, nil_to_add_test, random_state+10)
print(train_df_nil_unique_count_no_dev.loc[sample_transplant_to_test]['count'].sum(), '>=', nil_to_add_test)
assert train_df_nil_unique_count_no_dev.loc[sample_transplant_to_test]['count'].sum() >= nil_to_add_test

## Transplant NIL to dev and test

### Dev

In [ ]:
mentions_to_dev = train_df_nil.join(
    train_df_nil_unique_count.loc[sample_transplant_to_dev].set_index(['wikiId', 'word']),
    on=['wikiId', 'word'], how='inner')
mentions_to_dev = mentions_to_dev.drop(columns=['count'])

In [ ]:
dev_df_transplant = pd.concat([dev_df_merged, mentions_to_dev], ignore_index=True)
dev_df_transplant = dev_df_transplant.sample(frac=1, random_state=random_state) # randomize

In [ ]:
dev_df_transplant.shape

In [ ]:
print(dev_df_transplant.eval('NIL').sum(), '>=', desired_nil_total)
assert dev_df_transplant.eval('NIL').sum() >= desired_nil_total

### Test

In [ ]:
mentions_to_test = train_df_nil.join(
    train_df_nil_unique_count.loc[sample_transplant_to_test].set_index(['wikiId', 'word']),
    on=['wikiId', 'word'], how='inner')
mentions_to_test = mentions_to_test.drop(columns=['count'])

In [ ]:
test_df_transplant = pd.concat([test_df_merged, mentions_to_test], ignore_index=True)
test_df_transplant = test_df_transplant.sample(frac=1, random_state=random_state) # randomize

In [ ]:
test_df_transplant.shape

In [ ]:
print(test_df_transplant.eval('NIL').sum(), '>=', desired_nil_total)
assert test_df_transplant.eval('NIL').sum() >= desired_nil_total

### Remove from train

In [ ]:
train_df_transplant = train_df_merged.drop(mentions_to_dev.index.union(mentions_to_test.index))

In [ ]:
train_unique_transplant = train_df_transplant[['word', 'wikiId']].drop_duplicates(keep='first')

In [ ]:
dev_unique_transplant = dev_df_transplant[['word', 'wikiId']].drop_duplicates(keep='first')

In [ ]:
test_unique_transplant = test_df_transplant[['word', 'wikiId']].drop_duplicates(keep='first')

In [ ]:
# ensure no duplicates
assert not train_unique_transplant.duplicated().any()
assert not dev_unique_transplant.duplicated().any()
assert not test_unique_transplant.duplicated().any()

In [ ]:
# ensure unseen mentions holds
assert not pd.concat([train_unique_transplant, dev_unique_transplant]).duplicated().any()
assert not pd.concat([train_unique_transplant, test_unique_transplant]).duplicated().any()
assert not pd.concat([dev_unique_transplant, test_unique_transplant]).duplicated().any()

In [ ]:
print('Train', train_df_transplant.shape)
print('Dev', dev_df_transplant.shape)
print('Test', test_df_transplant.shape)

# Transplant not-NIL mentions

In [ ]:
# prendo n menzioni NIL dal train che non siano nel dev/test e ce le metto

In [ ]:
train_df_not_nil = train_df_merged.query('~NIL')

In [ ]:
desired_total = dev_test_desired_batch_size * n_batch

In [ ]:
not_nil_to_add_dev = desired_total - dev_df_transplant.shape[0]
not_nil_to_add_dev

In [ ]:
not_nil_to_add_test = desired_total - test_df_transplant.shape[0]
not_nil_to_add_test

In [ ]:
train_df_not_nil_unique_count = pd.DataFrame(
    train_df_not_nil[['wikiId', 'word']].value_counts(), columns=['count']).reset_index()
train_df_not_nil_unique_count.head()

In [ ]:
sample_transplant_to_dev_not_nil = get_subset_sum_ge(train_df_not_nil_unique_count, not_nil_to_add_dev, random_state)

In [ ]:
print(train_df_not_nil_unique_count.loc[sample_transplant_to_dev_not_nil]['count'].sum(), ">=", not_nil_to_add_dev)
assert train_df_not_nil_unique_count.loc[sample_transplant_to_dev_not_nil]['count'].sum() >= not_nil_to_add_dev

In [ ]:
# remove sample_transplant to dev before extracting a sample for test
train_df_not_nil_unique_count_no_dev = train_df_not_nil_unique_count.drop(sample_transplant_to_dev_not_nil)
sample_transplant_to_test_not_nil = get_subset_sum_ge(train_df_not_nil_unique_count_no_dev, not_nil_to_add_test, random_state+10)

In [ ]:
print(train_df_not_nil_unique_count_no_dev.loc[sample_transplant_to_test_not_nil]['count'].sum(), ">=", not_nil_to_add_test)
assert train_df_not_nil_unique_count_no_dev.loc[sample_transplant_to_test_not_nil]['count'].sum() >= not_nil_to_add_test

### Dev

In [ ]:
mentions_to_dev_not_nil = train_df_not_nil.join(
    train_df_not_nil_unique_count.loc[sample_transplant_to_dev_not_nil].set_index(['wikiId', 'word']),
    on=['wikiId', 'word'], how='inner')
mentions_to_dev_not_nil = mentions_to_dev_not_nil.drop(columns=['count'])

In [ ]:
assert not mentions_to_dev_not_nil.eval('NIL').any()

In [ ]:
dev_df_transplant_final = pd.concat([dev_df_transplant, mentions_to_dev_not_nil], ignore_index=True)
dev_df_transplant_final = dev_df_transplant_final.sample(frac=1, random_state=random_state) # randomize

In [ ]:
dev_df_transplant_final.shape

In [ ]:
assert dev_df_transplant_final.eval('NIL').sum() >= desired_nil_total
assert dev_df_transplant_final.shape[0] >= desired_total

### Test

In [ ]:
mentions_to_test_not_nil = train_df_not_nil.join(
    train_df_not_nil_unique_count.loc[sample_transplant_to_test_not_nil].set_index(['wikiId', 'word']),
    on=['wikiId', 'word'], how='inner')
mentions_to_test_not_nil = mentions_to_test_not_nil.drop(columns=['count'])

In [ ]:
test_df_transplant_final = pd.concat([test_df_transplant, mentions_to_test_not_nil], ignore_index=True)
test_df_transplant_final = test_df_transplant_final.sample(frac=1, random_state=random_state) # randomize

In [ ]:
test_df_transplant_final.shape

In [ ]:
assert test_df_transplant_final.eval('NIL').sum() >= desired_nil_total
assert test_df_transplant_final.shape[0] >= desired_total

### Remove from train

In [ ]:
train_df_transplant_final = train_df_transplant.drop(mentions_to_dev_not_nil.index.union(mentions_to_test_not_nil.index))

In [ ]:
train_unique_transplant_final = train_df_transplant_final[['word', 'wikiId']].drop_duplicates(keep='first')

In [ ]:
dev_unique_transplant_final = dev_df_transplant_final[['word', 'wikiId']].drop_duplicates(keep='first')

In [ ]:
test_unique_transplant_final = test_df_transplant_final[['word', 'wikiId']].drop_duplicates(keep='first')

In [ ]:
# ensure no duplicates
assert not train_unique_transplant_final.duplicated().any()
assert not dev_unique_transplant_final.duplicated().any()
assert not test_unique_transplant_final.duplicated().any()

In [ ]:
# ensure unseen mentions holds
assert not pd.concat([train_unique_transplant_final, dev_unique_transplant_final]).duplicated().any()
assert not pd.concat([train_unique_transplant_final, test_unique_transplant_final]).duplicated().any()
assert not pd.concat([dev_unique_transplant_final, test_unique_transplant_final]).duplicated().any()

In [ ]:
print('Train', train_df_transplant_final.shape)
print('Dev', dev_df_transplant_final.shape)
print('Test', test_df_transplant_final.shape)

# Remove samples from dev test to reach the desired number
If required, here it is possible to remove samples from the dev and test sets to reach a precise number of samples.

# Divide train and test in batches
Train is divided in batch similarly as it is in the original dataset to avoid a single very big file.

In [ ]:
skf = StratifiedKFold(n_splits=n_batch, shuffle=True, random_state=random_state)

# Train

In [ ]:
train_df_transplant_final = train_df_transplant_final.reset_index(drop=True)

In [ ]:
train_batch_indexes = []                                                                                                                                              
# the NIL class has freq = 0 so it is fairly distributed among the batches                                                                                            
for _, _index in skf.split(np.zeros(train_df_transplant_final.shape[0]), train_df_transplant_final['freq']):                                                          
    train_batch_indexes.append(_index)

In [ ]:
for i, batch in enumerate(train_batch_indexes):                                                                                                                       
    train_df_transplant_final.loc[batch, 'batch'] = i                                                                                                                 
train_df_transplant_final['batch'] = train_df_transplant_final['batch'].astype(int)

In [ ]:
train_df_transplant_final.head()

# Reset Dev index

In [ ]:
dev_df_transplant_final = dev_df_transplant_final.reset_index(drop=True)

## Test

In [ ]:
test_df_transplant_final = test_df_transplant_final.reset_index(drop=True)

In [ ]:
test_batch_indexes = []
# the NIL class has freq = 0 so it is fairly distributed among the batches
for _, _index in skf.split(np.zeros(test_df_transplant_final.shape[0]), test_df_transplant_final['freq']):
    test_batch_indexes.append(_index)

In [ ]:
for i, batch in enumerate(test_batch_indexes):
    test_df_transplant_final.loc[batch, 'batch'] = i
test_df_transplant_final['batch'] = test_df_transplant_final['batch'].astype(int)

In [ ]:
test_df_transplant_final.head()

# Prepare for BLINK

In [ ]:
train_df_transplant_final = train_df_transplant_final.rename(columns = {
    'left_context_text': 'context_left',
    'word': 'mention',
    'right_context_text': 'context_right',
    'ex_id': 'query_id',
    'url':'label_id',
    'wikiId':'Wikipedia_ID',
    'wikiurl':'Wikipedia_URL',
    'y_title': 'Wikipedia_title'
})

In [ ]:
dev_df_transplant_final = dev_df_transplant_final.rename(columns = {
    'left_context_text': 'context_left',
    'word': 'mention',
    'right_context_text': 'context_right',
    'ex_id': 'query_id',
    'url':'label_id',
    'wikiId':'Wikipedia_ID',
    'wikiurl':'Wikipedia_URL',
    'y_title': 'Wikipedia_title'
})

In [ ]:
test_df_transplant_final = test_df_transplant_final.rename(columns = {
    'left_context_text': 'context_left',
    'word': 'mention',
    'right_context_text': 'context_right',
    'ex_id': 'query_id',
    'url':'label_id',
    'wikiId':'Wikipedia_ID',
    'wikiurl':'Wikipedia_URL',
    'y_title': 'Wikipedia_title'
})

# Save to disk

In [ ]:
train_basedir = os.path.join(outdir, 'train')
os.makedirs(train_basedir, exist_ok=True)
print('Saving train...')
for batch in range(n_batch):
    print('Batch {} of {}'.format(batch + 1, n_batch))
    train_batch = train_df_transplant_final[train_df_transplant_final['batch'] == batch]
    with open(os.path.join(train_basedir, 'train_{}.jsonl'.format(batch)), 'w') as fd:
        for i, row in tqdm(train_batch.iterrows(), total=train_batch.shape[0]):
            fd.write(row.to_json())
            fd.write('\n')

In [ ]:
dev_basedir = os.path.join(outdir, 'dev')
os.makedirs(dev_basedir, exist_ok=True)
print('Saving dev...')
with open(os.path.join(dev_basedir, 'dev.jsonl'), 'w') as fd:
    for i, row in tqdm(dev_df_transplant_final.iterrows(), total=dev_df_transplant_final.shape[0]):
        fd.write(row.to_json())
        fd.write('\n')

In [ ]:
test_basedir = os.path.join(outdir, 'test')
os.makedirs(test_basedir, exist_ok=True)
print('Saving test...')
for batch in range(n_batch):
    print('Batch {} of {}'.format(batch + 1, n_batch))
    test_batch = test_df_transplant_final[test_df_transplant_final['batch'] == batch]
    with open(os.path.join(test_basedir, 'test_{}.jsonl'.format(batch)), 'w') as fd:
        for i, row in tqdm(test_batch.iterrows(), total=test_batch.shape[0]):
            fd.write(row.to_json())
            fd.write('\n')

# Statistics

In [ ]:
report = []

In [ ]:
for batch in trange(n_batch):
    test_batch = test_df_transplant_final[test_df_transplant_final['batch'] == batch]
    
    batch_report = {}
    batch_report['batch'] = batch
    
    # mentions
    batch_report['test_mentions'] = test_batch.shape[0]

    # entities
    batch_report['test_entities'] = test_batch['Wikipedia_ID'].drop_duplicates().shape[0]

    # NIL mentions
    batch_report['test_nil_mentions'] = test_batch.eval('NIL').sum()
    
    # not-NIL mentions
    batch_report['test_not_nil_mentions'] = batch_report['test_mentions'] - batch_report['test_nil_mentions']

    # NIL entities
    batch_report['test_nil_entities'] = test_batch.query('NIL')['Wikipedia_ID'].drop_duplicates().shape[0]
    
    # not-NIL entities
    batch_report['test_not_nil_entities'] = batch_report['test_entities'] - batch_report['test_nil_entities']
    
    # NIL entities present in a previous batch
    test_indexes = set(test_batch.query('NIL')['Wikipedia_ID'].drop_duplicates()
           ).intersection(
            # the set of all the nil entities found in the previous batches
            set(test_df_transplant_final.query('NIL and batch < {}'.format(batch))['Wikipedia_ID'].drop_duplicates())
        )
    batch_report['test_nil_entities_found_in_previous_batch'] = len(test_indexes)
    
    # NIL mentions present in a previous batch
    batch_report['test_nil_mentions_found_in_previous_batch'] = test_batch['Wikipedia_ID'].isin(test_indexes).sum()
    
    report.append(batch_report)

In [ ]:
all_dataset = {}
all_dataset['batch'] = 'ALL'

# mentions
all_dataset['train_mentions'] = train_df_transplant_final.shape[0]
all_dataset['dev_mentions'] = dev_df_transplant_final.shape[0]
all_dataset['test_mentions'] = test_df_transplant_final.shape[0]

# entities
all_dataset['train_entities'] = train_df_transplant_final['Wikipedia_ID'].drop_duplicates().shape[0]
all_dataset['dev_entities'] = dev_df_transplant_final['Wikipedia_ID'].drop_duplicates().shape[0]
all_dataset['test_entities'] = test_df_transplant_final['Wikipedia_ID'].drop_duplicates().shape[0]

# NIL mentions
all_dataset['train_nil_mentions'] = train_df_transplant_final.eval('NIL').sum()
all_dataset['dev_nil_mentions'] = dev_df_transplant_final.eval('NIL').sum()
all_dataset['test_nil_mentions'] = test_df_transplant_final.eval('NIL').sum()

# not-NIL mentions
all_dataset['train_not_nil_mentions'] = all_dataset['train_mentions'] - all_dataset['train_nil_mentions']
all_dataset['dev_not_nil_mentions'] = all_dataset['dev_mentions'] - all_dataset['dev_nil_mentions']
all_dataset['test_not_nil_mentions'] = all_dataset['test_mentions'] - all_dataset['test_nil_mentions']

# NIL entities
all_dataset['train_nil_entities'] = train_df_transplant_final.query('NIL')['Wikipedia_ID'].drop_duplicates().shape[0]
all_dataset['dev_nil_entities'] = dev_df_transplant_final.query('NIL')['Wikipedia_ID'].drop_duplicates().shape[0]
all_dataset['test_nil_entities'] = test_df_transplant_final.query('NIL')['Wikipedia_ID'].drop_duplicates().shape[0]

# not-NIL entities
all_dataset['train_not_nil_entities'] = all_dataset['train_entities'] - all_dataset['train_nil_entities']
all_dataset['dev_not_nil_entities'] = all_dataset['dev_entities'] - all_dataset['dev_nil_entities']
all_dataset['test_not_nil_entities'] = all_dataset['test_entities'] - all_dataset['test_nil_entities']

# NIL entities present in a previous batch
all_dataset['train_nil_entities_found_in_previous_batch'] = 0
all_dataset['dev_nil_entities_found_in_previous_batch'] = 0
all_dataset['test_nil_entities_found_in_previous_batch'] = 0

# NIL mentions present in a previous batch
all_dataset['train_nil_mentions_found_in_previous_batch'] = 0
all_dataset['dev_nil_mentions_found_in_previous_batch'] = 0
all_dataset['test_nil_mentions_found_in_previous_batch'] = 0

report.append(all_dataset)

In [ ]:
original_dataset = {}
original_dataset['batch'] = 'ORIGINAL'

# mentions
original_dataset['train_mentions'] = train_df.shape[0]
original_dataset['dev_mentions'] = dev_df.shape[0]
original_dataset['test_mentions'] = test_df.shape[0]

# entities
original_dataset['train_entities'] = train_df['wikiId'].drop_duplicates().shape[0]
original_dataset['dev_entities'] = dev_df['wikiId'].drop_duplicates().shape[0]
original_dataset['test_entities'] = test_df['wikiId'].drop_duplicates().shape[0]

# NIL mentions
original_dataset['train_nil_mentions'] =  0
original_dataset['dev_nil_mentions'] = 0
original_dataset['test_nil_mentions'] = 0

# not-NIL mentions
original_dataset['train_not_nil_mentions'] = original_dataset['train_mentions'] - original_dataset['train_nil_mentions']
original_dataset['dev_not_nil_mentions'] = original_dataset['dev_mentions'] - original_dataset['dev_nil_mentions']
original_dataset['test_not_nil_mentions'] = original_dataset['test_mentions'] - original_dataset['test_nil_mentions']

# NIL entities
original_dataset['train_nil_entities'] = 0
original_dataset['dev_nil_entities'] = 0
original_dataset['test_nil_entities'] = 0

# not-NIL entities
original_dataset['train_not_nil_entities'] = original_dataset['train_entities'] - original_dataset['train_nil_entities']
original_dataset['dev_not_nil_entities'] = original_dataset['dev_entities'] - original_dataset['dev_nil_entities']
original_dataset['test_not_nil_entities'] = original_dataset['test_entities'] - original_dataset['test_nil_entities']

# NIL entities present in a previous batch
original_dataset['train_nil_entities_found_in_previous_batch'] = 0
original_dataset['dev_nil_entities_found_in_previous_batch'] = 0
original_dataset['test_nil_entities_found_in_previous_batch'] = 0

# NIL mentions present in a previous batch
original_dataset['train_nil_mentions_found_in_previous_batch'] = 0
original_dataset['dev_nil_mentions_found_in_previous_batch'] = 0
original_dataset['test_nil_mentions_found_in_previous_batch'] = 0

report.append(original_dataset)

In [ ]:
report_df = pd.DataFrame(report)
report_df

## Save statistics

In [ ]:
stats_basedir = os.path.join(outdir, 'statistics')
os.makedirs(stats_basedir, exist_ok=True)

In [ ]:
report_df.to_csv(os.path.join(stats_basedir, 'statistics.csv'))

 # Remove NIL entities from the KB

In [ ]:
NIL_entities = []
NIL_entities.extend(
    train_df_transplant_final.query('NIL')['Wikipedia_ID'].tolist())
NIL_entities.extend(
    dev_df_transplant_final.query('NIL')['Wikipedia_ID'].tolist())
NIL_entities.extend(
    test_df_transplant_final.query('NIL')['Wikipedia_ID'].tolist())

In [ ]:
NIL_entities = set(NIL_entities) # remove duplicates

In [ ]:
sql_list = ",".join(str(i) for i in NIL_entities)
indexer = 10 # ensure to put the correct indexer
sql_query = 'DELETE FROM entities WHERE indexer = {} AND wikipedia_id in ({});'.format(indexer, sql_list)

In [ ]:
with open(os.path.join(outdir, 'delete_nil_entities.sql'), 'w') as fd:
    fd.write(sql_query + '\n')

Now run the sql query in the database (e.g.):
```
sudo docker-compose exec -T postgres psql -U postgres < /path/to/incremental_dataset/delete_nil_entities.sql
```

At this point NIL entities are removed from the db and even if retrieved by the faiss indexer they are discarded.